# QAOA Pulser

In [1]:
import numpy as np
import json

import matplotlib.pylab as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.cm as cm
import networkx as nx
from collections import defaultdict

from pulser import Pulse, Sequence, Register
from pulser.waveforms import CompositeWaveform, ConstantWaveform, RampWaveform
from pulser.devices import AnalogDevice, DigitalAnalogDevice

from pulser_simulation import SimConfig, QutipEmulator

from pulser.json.abstract_repr.deserializer import deserialize_device

from pasqal_cloud import SDK

In [2]:
class NumpyEncoder(json.JSONEncoder):
    """ Custom encoder for numpy data types """
    def default(self, obj):
        if isinstance(obj, ( np.int64)):
            return int(obj)      
        return json.JSONEncoder.default(self, obj)

In [3]:
file1 = open('./project_id.txt', 'r')
file2 = open('./username.txt','r')
file3 = open('./pass.txt', 'r')

project_id = file1.read()
username = file2.read()
password = file3.read()
file1.close()
file2.close()
file3.close()

# Initialize the cloud client
sdk = SDK(username=username, project_id=project_id,password=password)

specs = sdk.get_device_specs_dict()
device = deserialize_device(specs["FRESNEL"])

In [4]:
device_emu = AnalogDevice

In [5]:
def cost_function(x, G):
    obj = 0
    for i, j in G.edges():
        if x[i] + x[j] == "11":
            obj += 2
    return - x.count("1") + obj

In [6]:
backends = {}
backends["pasqal_fresnel"] = device
backends["pulser_emulator"] = device_emu
backends["pulser_emulator_noisy"] = device_emu

In [7]:
# Prepare Pulse (Independent of the problem size)

with open("./Data/opt_schedule.json", "r") as file:
    schedule = json.load(file)

omega_opt_list = [i for i in schedule["omega_list"]]
delta_opt_list = [i for i in schedule["delta_list"]]
omega_opt_list = np.array(omega_opt_list)*1e-6
delta_opt_list = np.array(delta_opt_list)*1e-6
num_pulses = len(schedule["omega_list"])

t_tot = 2000
t_q = 52
t_p = int(t_tot/num_pulses-t_q)
t_step = t_q+t_p
t_delay = 1952

pulse = Pulse(
                CompositeWaveform(
                    RampWaveform(t_q,0.,omega_opt_list[0]),
                    ConstantWaveform(t_p,omega_opt_list[0]),
                    RampWaveform(t_q,omega_opt_list[0],omega_opt_list[1]),
                    ConstantWaveform(t_p,omega_opt_list[1]),
                    RampWaveform(t_q,omega_opt_list[1],omega_opt_list[2]),
                    ConstantWaveform(t_p,omega_opt_list[2]),
                    RampWaveform(t_q,omega_opt_list[2],omega_opt_list[3]),
                    ConstantWaveform(t_p,omega_opt_list[3]),
                    RampWaveform(t_q,omega_opt_list[3],omega_opt_list[4]),
                    ConstantWaveform(t_p,omega_opt_list[4]),
                    RampWaveform(t_q,omega_opt_list[4],omega_opt_list[5]),
                    ConstantWaveform(t_p,omega_opt_list[5]),
                    RampWaveform(t_q,omega_opt_list[5],omega_opt_list[6]),
                    ConstantWaveform(t_p,omega_opt_list[6]),
                    RampWaveform(t_q,omega_opt_list[6],omega_opt_list[7]),
                    ConstantWaveform(t_p,omega_opt_list[7]),
                    RampWaveform(t_q,omega_opt_list[7],omega_opt_list[8]),
                    ConstantWaveform(t_p,omega_opt_list[8]),
                    RampWaveform(t_q,omega_opt_list[8],omega_opt_list[9]),
                    ConstantWaveform(t_p,omega_opt_list[9]),
                    RampWaveform(t_q,omega_opt_list[9],0.),
                ),
                CompositeWaveform(
                    RampWaveform(t_q,0.,delta_opt_list[0]),
                    ConstantWaveform(t_p,delta_opt_list[0]),
                    RampWaveform(t_q,delta_opt_list[0],delta_opt_list[1]),
                    ConstantWaveform(t_p,delta_opt_list[1]),
                    RampWaveform(t_q,delta_opt_list[1],delta_opt_list[2]),
                    ConstantWaveform(t_p,delta_opt_list[2]),
                    RampWaveform(t_q,delta_opt_list[2],delta_opt_list[3]),
                    ConstantWaveform(t_p,delta_opt_list[3]),
                    RampWaveform(t_q,delta_opt_list[3],delta_opt_list[4]),
                    ConstantWaveform(t_p,delta_opt_list[4]),
                    RampWaveform(t_q,delta_opt_list[4],delta_opt_list[5]),
                    ConstantWaveform(t_p,delta_opt_list[5]),
                    RampWaveform(t_q,delta_opt_list[5],delta_opt_list[6]),
                    ConstantWaveform(t_p,delta_opt_list[6]),
                    RampWaveform(t_q,delta_opt_list[6],delta_opt_list[7]),
                    ConstantWaveform(t_p,delta_opt_list[7]),
                    RampWaveform(t_q,delta_opt_list[7],delta_opt_list[8]),
                    ConstantWaveform(t_p,delta_opt_list[8]),
                    RampWaveform(t_q,delta_opt_list[8],delta_opt_list[9]),
                    ConstantWaveform(t_p,delta_opt_list[9]),
                    ConstantWaveform(t_q,delta_opt_list[9]),
                ),
                phase=0.
)

In [27]:
# Prepare Register

nq =  44# can do until 51
with open(f"./Data/Problems/{nq}.json", "r") as file:
    problem = json.load(file)

G = nx.Graph()
G.add_nodes_from(range(nq))
G.add_edges_from(problem["edges"])
problem.keys()
a = 5.
grid_side = problem["grid_side"]
pos = problem["pos"]
numx,numy = [grid_side,grid_side]
x_shift = a * (numx - 1) / 2
y_shift = a * (numy - 1) / 2
pos_dict = {}
for i, pos_i in enumerate(pos):
    pos_dict[f"q{i}"] = [round(pos_i[0] * a - x_shift,8), round(pos_i[1] * a - y_shift,8)]

register = Register(pos_dict)

# register.draw(with_labels=True,
#           blockade_radius=2**(3/4)*a,
#           draw_graph=True,
#           draw_half_radius=True,
#           qubit_colors={},
#           fig_name=None,#f"./../../Plots/reg2.png",
#           kwargs_savefig={},
#           custom_ax=None,
#           show=True)

In [29]:
# Run experiment

if nq > 10 and nq < 21:
    shots = 400
if nq > 20 and nq < 31:
    shots = 600
if nq > 30 and nq < 41:
    shots = 800
if nq > 40:
    shots = 1000

shots = 200
result = {}
backend_name = ["pulser_emulator", "pulser_emulator_noisy", "pasqal_fresnel"][2]

if backend_name == "pasqal_fresnel":
    register = register.with_automatic_layout(backends[backend_name])

sequence = Sequence(register, backends[backend_name])
sequence.declare_channel('global_fields', 'rydberg_global')
sequence.delay(t_delay,'global_fields')
sequence.add(pulse,'global_fields')
#sequence.draw()

if backend_name == "pulser_emulator":
    config = SimConfig()
if backend_name == "pulser_emulator_noisy":
    config = SimConfig(noise=("SPAM"), eta=0.02,epsilon=0.01,epsilon_prime=0.08,runs=30, samples_per_run=5)

if backend_name in ["pulser_emulator", "pulser_emulator_noisy"]:
    emu = QutipEmulator.from_sequence(sequence, sampling_rate = 0.05, evaluation_times=0.1,config=config).run()
    #job = QutipBackend(sequence, config = config).run()
    result["samples"] = emu.sample_final_state(N_samples=shots)
    with open(f"./Data/{backend_name}/{nq}.json", "w") as file:
        json.dump(result,file,cls=NumpyEncoder)
if backend_name == "pasqal_fresnel":
    serialized_sequence = sequence.to_abstract_repr()
    job = {"job_ID":f"nq{nq}","runs": shots}
    batch = sdk.create_batch(serialized_sequence, [job]#, emulator=EmulatorType.EMU_FREE
                            )

RuntimeError: Failed to find a site for 7 traps.

In [14]:
# Postprocessing

nq = 21
backend_name = ["pulser_emulator", "pulser_emulator_noisy", "pasqal_fresnel"][2]
with open(f"./Data/{backend_name}/{nq}.json", "r") as file:
    result = json.load(file)
if backend_name == "pasqal_fresnel":
    batch_id = list(result.keys())[0]
    result = result[batch_id]
    result['samples'] = result.pop('counter')
    

with open(f"./Data/problems/{nq}.json", "r") as file:
    problem = json.load(file)
G = nx.Graph()
G.add_nodes_from(range(nq))
G.add_edges_from(problem["edges"])

cost_evals = defaultdict(int)
for k, v in result["samples"].items():
    cost = cost_function(k, G)
    cost_evals[cost] += v
result["cost"] = cost_evals 
result["min_cost"] = cost_function(problem["sol"], G)

with open(f"./Data/{backend_name}/{nq}.json", "w") as file:
    json.dump(result, file)

FileNotFoundError: [Errno 2] No such file or directory: './Data/pasqal_fresnel/21.json'

In [24]:
sum(result['samples'].values())

200